In [1]:
%matplotlib inline
import pandas as pd
import sklearn as skl
import tensorflow as tf
import numpy as np
from pathlib import Path

import datetime
import time

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Run me second

In [2]:
cleaned_df = pd.read_csv('./Resources/ealtman2019/cleaned_columns.csv')
cleaned_df.head()

MemoryError: Unable to allocate 662. MiB for an array with shape (4, 21707289) and data type int64

In [ ]:
list(cleaned_df.columns)

In [ ]:
sample_df = cleaned_df.sample(n=100000, random_state = 78)
sample_df.shape

In [ ]:
cols=["Year","Month","Day", "Time"]

sample_df['Date'] = sample_df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")


In [ ]:
sample_df['Date']=pd.to_datetime(sample_df['Date'])

In [ ]:
sample_df = sample_df.drop(columns= cols)

In [ ]:
sample_df = sample_df.drop(columns= ['Merchant City', 'Merchant State'])

In [ ]:
sample_df['Errors?'].nunique()

In [ ]:
list(sample_df.columns)

In [ ]:
# sample_df['Date'].head()

In [ ]:
sample_df['Date'] = sample_df['Date'].astype(np.int64) // 10**9

In [ ]:
sample_df.head()

In [ ]:
sample_df = pd.get_dummies(sample_df, columns =['Errors?'])

In [ ]:
list(sample_df.columns)

In [ ]:
sample_df = sample_df.rename(columns={
									    'Errors?_Bad CVV':"Bad CVV",
										 'Errors?_Bad CVV,Insufficient Balance':"Bad CVV,Insufficient Balance",
										 'Errors?_Bad CVV,Technical Glitch':"Bad CVV,Technical Glitch",
										 'Errors?_Bad Card Number':"Bad Card Number",
										 'Errors?_Bad Card Number,Bad CVV':"Bad Card Number,Bad CVV",
										 'Errors?_Bad Card Number,Bad Expiration':"Bad Card Number,Bad Expiration",
										 'Errors?_Bad Card Number,Insufficient Balance':"Bad Card Number,Insufficient Balance",
										 'Errors?_Bad Card Number,Technical Glitch':"Bad Card Number,Technical Glitch",
										 'Errors?_Bad Expiration':"Bad Expiration",
										 'Errors?_Bad Expiration,Bad CVV':"Bad Expiration,Bad CVV",
										 'Errors?_Bad Expiration,Insufficient Balance':"Bad Expiration,Insufficient Balance",
										 'Errors?_Bad PIN':"Bad PIN",
										 'Errors?_Bad PIN,Insufficient Balance':"Bad PIN,Insufficient Balance",
										 'Errors?_Bad PIN,Technical Glitch':"Bad PIN,Technical Glitch",
										 'Errors?_Bad Zipcode':"Bad Zipcode",
										 'Errors?_Insufficient Balance':"Insufficient Balance",
										 'Errors?_Insufficient Balance,Technical Glitch':"Insufficient Balance,Technical Glitch",
										 'Errors?_Technical Glitch':"Technical Glitch"
									  })

In [ ]:
list(sample_df.columns)

In [ ]:
sample_df = sample_df.drop(columns=['Errors?_0'])

In [ ]:
list(sample_df.columns)

In [ ]:
sample_df.head()

In [ ]:
sample_df.dtypes

In [ ]:
# Split the data into X_train, X_test, y_train, y_test
y = sample_df["Is Fraud?"].values
X = sample_df.drop("Is Fraud?", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train a Logistic Regression model print the model score
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
tsne = TSNE(learning_rate=50, n_components=3, perplexity=50, verbose=1, n_iter=250)

In [ ]:
tsne_features = tsne.fit_transform(sample_df)

In [ ]:
tsne_features.shape

In [ ]:
labels = sample_df['Is Fraud?']
plt.scatter(tsne_features[:,0],tsne_features[:,1],c=labels)

In [ ]:
# Create an elbow plot to identify the best number of clusters.
inertia = []
k = list(range(1, 11))

# Calculate the inertia
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(sample_df)
    inertia.append(km.inertia_)

    
# 3.2 If possible, determine where the elbow of the plot is, and at which value of k it appears.
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.head()

In [ ]:
# Plot the elbow curve to find the best candidate(s) for k
plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(range(1,11))
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Fraud status elbow curve')
plt.show()

In [ ]:
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [ ]:
features = clf.feature_importances_
print(features)
plt.bar(x = range(len(features)), height=features)
plt.xlabel('Transaction Types')
plt.show()

In [ ]:
# The most relevent features are Merchant Type and User, with the mosdt common area of fraud being the Middle East
# The only relevant error type is Bad Zipcode, which most primarily relates to online or automated transactions
